# AgriWeather

## 관측지점정보

- 주의사항 : 모든 관측지점정보가 있는 것은 아님. ex) 강릉시 연곡면, 210852A001

- Data Source : [관측지점정보 - 농업기상정보](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwjh9Iz52I38AhVDEIgKHWFJDNMQFnoECA4QAQ&url=http%3A%2F%2Fweather.rda.go.kr%2Fw%2Fweather%2FobservationInfo.do&usg=AOvVaw2YyGtn4LBYHLzmmtv-AsXB)

- Collection Method : Download

- Data Format : CSV

### 데이터 설명

- '**농업기상 기본 관측데이터 조회**'에 활용할 '**관측지점코드**'를 얻기 위함.

---

## 농촌진흥청 국립농업과학원\_농업기상 기본 관측데이터 조회

- Data Source : [농촌진흥청 국립농업과학원\_농업기상 기본 관측데이터 조회](https://www.data.go.kr/data/15078057/openapi.do) → 농업기상 조회년도별 일 기본 관측데이터

- Collection Method : API

- Data Format : XML

### 데이터 설명

- 농업기상 관측데이터 (**기온, 습도, 풍향, 풍속, 강수량, 일조시간, 일사량 등**)을 정보를 제공합니다.

### 요청변수(Request Parameter)

| 항목명(국문)      | 항목명(영문)   | 항목크기 | 항목구분 | 샘플데이터 | 항목설명                              |
| ----------------- | -------------- | -------- | -------- | ---------- | ------------------------------------- |
| 인증키            | serviceKey     | 100      | 필       | 인증키     | 공공데이터포털에서 발급받은 인증키    |
| 페이지 번호       | Page_No        | 10       | 필       | 1          | 페이지 번호                           |
| 한 페이지 결과 수 | Page_Size      | 10       | 필       | 20         | 한 페이지 결과 수 (1 ~ 100 최대 허용) |
| 관측년도          | search_Year    | 4        | 필       | 2018       | 관측년도                              |
| 관측지점코드      | obsr_Spot_Code | 10       | 필       | 210852A001 | 관측지점코드                          |

### 출력결과(Response Element)

| 항목명(국문)      | 항목명(영문) | 항목크기 | 항목구분 | 샘플데이터       | 항목설명                  |
| ----------------- | ------------ | -------- | -------- | ---------------- | ------------------------- |
| 결과코드          | result_Code  | 3        | 필       | 200              | 결과코드                  |
| 결과메세지        | result_Msg   | 50       | 필       | OK               | 결과메시지                |
| 한 페이지 결과 수 | rcdcnt       | 4        | 필       | 20               | 한 페이지 결과 수         |
| 페이지 번호       | page_No      | 4        | 필       | 1                | 페이지 번호               |
| 전체 결과 수      | total_Count  | 4        | 필       | 29117            | 전체 결과 수              |
| 번호              | no           | 10       | 필       | 1                | 데이터 검색결과 정렬 번호 |
| 관측지점코드      | stn_Code     | 10       | 필       | 477802A001       | 관측지점코드              |
| 관측지점명        | stn_Name     | 30       | 필       | 가평군 가평읍    | 관측지점명                |
| 관측시각          | date         | 16       | 필       | 2018-01-01 00:10 | 관측시각                  |
| 기온              | temp         | 12       | 필       | 2.1              | 기온(℃)                   |
| 최고기온          | max_Temp     | 12       | 필       | -1.9             | 최고기온(℃)               |
| 최저기온          | min_Temp     | 12       | 필       | -2.5             | 최저기온(℃)               |
| 습도              | hum          | 12       | 필       | 41.6             | 습도(%)                   |
| 풍향              | widdir       | 12       | 필       | 172.8            | 풍향                      |
| 풍속              | wind         | 12       | 필       | 1                | 풍속(m/s)                 |
| 강수량            | rain         | 12       | 필       | 0                | 강수량(mm)                |
| 일조시간          | sun_Time     | 12       | 필       | 227              | 일조시간(MM)              |
| 일사량            | sun_Qy       | 12       | 필       | 0                | 일사량(MJ/m²)             |
| 결로시간          | condens_Time | 12       | 필       | 0                | 결로시간(MM)              |
| 초상온도          | gr_Temp      | 12       | 필       | -3.8             | 초상온도(℃)               |
| 지중온도          | soil_Temp    | 12       | 필       | -0.6             | 지중온도(℃)               |
| 토양수분보정값    | soil_Wt      | 12       | 필       | 13               | 토양수분보정값(%)         |


# 회고

- **농업기상 기본 관측데이터 조회**'에 활용할 '**관측지점코드**'를 구하는 것이 어려웠다.
  - API에서 제공하는 모든 관측지점의 '**관측지점코드**'를 얻지 못하였다.
- xml형식의 데이터를 다루는 것이 어려웠다.


In [42]:
# import os
# import glob
import time
# from pprint import pprint

import numpy as np
import pandas as pd

# DC
import requests
from bs4 import BeautifulSoup

# about file format
# from io import BytesIO
# from zipfile import ZipFile
# import json
# import xml.etree.ElementTree as et
import xmltodict

# 'auth' is included in gitignore.
import sys
sys.path.append("../Import")
import auth
authkey = auth.authkey["AgriWeather"]


dfname = {
    "no": "No",
    "stn_Code": "ObserveSpot_Code",
    "stn_Name": "ObserveSpot_Name",
    "date":	"Date",
    "temp": "Temp",
    "max_Temp": "Max_Temp",
    "min_Temp": "Min_Temp",
    "hum":'humidity',
    "widdir":'Wind_Direction',
    "wind":'Wind_Speed',
    "rain":'Rainfall',
    "sun_Time":'Sun_Time',
    "sun_Qy":'Sun_Quantity',
    "condens_Time": "Condens_Time",
    "gr_Temp":  'Grass_Temp',
    "soil_Temp": "Soil_Temp",
    "soil_Wt": "Soil_Moisture_CorrValue"
    }


list(dfname.keys())

# 지점명 → 지점코드

- <u>**지점명 → 지점코드**</u> → 농업기상정보

- 관측지점 목록 만들기


In [2]:
# '관측지점정보.csv'를 불러오고, Pandas.DataFrame으로 만들기.
df_ObsrSpot = pd.read_csv("../data/관측지점정보.csv", encoding="cp949", index_col=False)

print("┌▣ 'df_ObsrSpot'의 raw")
display(df_ObsrSpot.head())


# 전처리
df_ObsrSpot["도명+지점명"] = df_ObsrSpot["도명"] + " " + df_ObsrSpot["지점명"]
df_ObsrSpot = df_ObsrSpot.drop(columns=["도명", '위도', '경도', '고도', "관측시작일"])

# 컬럼 순서 변경
# df_ObsrSpot.columns.to_list()
# ['지점명', '지점코드', '위도', '경도', '고도', '설치주소', '도명+지점명']
df_ObsrSpot = df_ObsrSpot[['도명+지점명', '지점명', '설치주소', '지점코드']]

print("\n┌▣ 'df_ObsrSpot'의 raw에서 전처리")
display(df_ObsrSpot.head())

print("\n┌▣ '도명+지점명'과 '지점명'의 중복 여부 확인 (중복값이 없어야함)")
display(df_ObsrSpot.describe(include = "O"))

┌▣ 'df_ObsrSpot'의 raw


,도명,지점명,지점코드,위도,경도,고도,설치주소,관측시작일
0,경기도,가평군 가평읍,477802A001,37.84621,127.50063,80.0,경기도 가평군 가평읍 아랫마장길 59,2011-05-22
1,경기도,고양시 구산동,411801A001,37.67453,126.70070,24.0,경기도 고양시 일산서구 구산동 1942,2010-12-06
2,경기도,고양시 덕양구,412040A002,37.64918,126.87036,39.0,경기도 고양시 덕양구 고양대로 1695 (원흥동),2013-08-24
3,경기도,광주시 목현동,464030A001,37.43231,127.23394,91.0,경기도 광주시 이배재로 209-5,2010-11-01
4,경기도,김포시 월곶면,415743A001,37.69489,126.55614,43.0,경기도 김포시 월곶면 오리정로 13,2010-12-14



┌▣ 'df_ObsrSpot'의 raw에서 전처리


,도명+지점명,지점명,설치주소,지점코드
0,경기도 가평군 가평읍,가평군 가평읍,경기도 가평군 가평읍 아랫마장길 59,477802A001
1,경기도 고양시 구산동,고양시 구산동,경기도 고양시 일산서구 구산동 1942,411801A001
2,경기도 고양시 덕양구,고양시 덕양구,경기도 고양시 덕양구 고양대로 1695 (원흥동),412040A002
3,경기도 광주시 목현동,광주시 목현동,경기도 광주시 이배재로 209-5,464030A001
4,경기도 김포시 월곶면,김포시 월곶면,경기도 김포시 월곶면 오리정로 13,415743A001



┌▣ '도명+지점명'과 '지점명'의 중복 여부 확인 (중복값이 없어야함)


,도명+지점명,지점명,설치주소,지점코드
count,211,211,211,211
unique,211,211,211,210
top,경기도 가평군 가평읍,가평군 가평읍,경기도 가평군 가평읍 아랫마장길 59,2.33E+06
freq,1,1,1,2


In [3]:
def Get_ObsrSpotCode(dataframe, obsr_Spot_Name) -> str:

    '''
    매개변수 'dataframe'에서 column '도명+지점명', '지점명', '설치주소' 중에서 
    매개변수 'obsr_Spot_Name'과 일치하는 row의 '지점코드'column의 value를 반환하는 함수
    '''

    # [0] - '도명+지점명'과 '지점명'의 중복 여부 확인했을 때, 중복이 없다는 전제로 슬라이싱을 활용함.

    r = dataframe[(dataframe["도명+지점명"] == obsr_Spot_Name) |(dataframe["지점명"] == obsr_Spot_Name) | (dataframe["설치주소"] == obsr_Spot_Name)]["지점코드"]

    return r.values[0]


# fnc test
Get_ObsrSpotCode(dataframe=df_ObsrSpot, obsr_Spot_Name = "광주시 목현동")

'464030A001'

# 지점코드 → 농업기상정보

- 지점명 → <u>**지점코드 → 농업기상정보**</u>


In [61]:
# api test

url = 'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherYearDayList'
params ={'serviceKey' : authkey, 'Page_No' : '1', 'Page_Size' : '20', 'search_Year' : '2018', 'obsr_Spot_Code' : '210852A001' }

time.sleep(0.2)
response = requests.get(url, params=params)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "xml")
    soup = soup.find_all("item")[0]

    parsed_dict = xmltodict.parse(str(soup))

display(pd.DataFrame(parsed_dict).T.reset_index(drop=True))
print(pd.DataFrame(parsed_dict).T.columns.to_list())

columns = list(parsed_dict["item"].keys())
columns.remove("no")
print(columns)

IndexError: list index out of range

In [ ]:
def Get_AgriWeather(authkey, columns, Page_No, search_Year, obsr_Spot_Code):
    
        url = f'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherYearDayList?serviceKey={authkey}&Page_No={Page_No}&Page_Size=100&search_Year={search_Year}&obsr_Spot_Code={obsr_Spot_Code}'

        time.sleep(0.2)
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'xml')
            soup = soup.find_all("item")

        df = pd.DataFrame()

        for i in range(0,100):
            s = soup[i]
            temp_dict = {"temp": {}}

            for j in columns:
                v = s.find(j).get_text()
                temp_dict["temp"][j] = v

            df_new = pd.DataFrame(temp_dict).T

        return df.append(df_new, ignore_index=True)[columns]


display(Get_AgriWeather(authkey=authkey, columns=columns, Page_No=1, search_Year=2018, obsr_Spot_Code="210852A001"))

In [71]:
def Get_All_AgriWeather(authkey=authkey, columns=columns, search_Year=(2005, 2020), obsr_Spot_Code="210852A001"):

    def Get_AgriWeather(authkey, columns, Page_No, search_Year, obsr_Spot_Code):
        
            url = f'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherYearDayList?serviceKey={authkey}&Page_No={Page_No}&Page_Size=100&search_Year={search_Year}&obsr_Spot_Code={obsr_Spot_Code}'

            time.sleep(0.2)
            response = requests.get(url)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'xml')
                soup = soup.find_all("item")

            df = pd.DataFrame()

            for i in range(0,100):
                s = soup[i]
                temp_dict = {"temp": {}}

                for j in columns:
                    v = s.find(j).get_text()
                    temp_dict["temp"][j] = v

                df_new = pd.DataFrame(temp_dict).T

            return df.append(df_new, ignore_index=True)[columns]
    
    for year in range(search_Year[0], search_Year[1]+1):
         
         Get_AgriWeather


         
    

[1]